# پیش پردازش متون فارسی با استفاده از فارسی‌یار

In [ ]:
#https://app.text-mining.ir/CustomerPanel

In [ ]:
import requests
import json

def callApi(url, data, tokenKey):
    headers = {
        'Content-Type': "application/json",
        'Authorization': "Bearer " + tokenKey,
        'Cache-Control': "no-cache"
    }
    response = requests.request("POST", url, data=data.encode("utf-8"), headers=headers)
    return response.text

##################### Get Token by Api Key ##########################


baseUrl = "http://api.text-mining.ir/api/"
url = baseUrl + "Token/GetToken"
querystring = {"apikey":"51f75c8d-367d-eb11-80ee-98ded002619b"}
response = requests.request("GET", url, params=querystring)
data = json.loads(response.text)
tokenKey = data['token']
#print (tokenKey)

# (Normalizer) نرمال ساز متن

In [ ]:
baseUrl = "http://api.text-mining.ir/api/"
url =  baseUrl + "PreProcessing/NormalizePersianWord"
text = "ك - ک - می خواهم - ایده ی خوب"
payload = u"{\"text\":\""+text+"\", \"refineSeparatedAffix\":true}"
print(callApi(url, payload, tokenKey))

ک- ک- می‌خواهم- ایده‌ی خوب


# (Sentence Splitter and Tokenizer) تقطیع جملات و واژه ها

In [ ]:
url =  baseUrl + "PreProcessing/SentenceSplitterAndTokenize"
payload = u'''{\"text\": \"او دانشجو است. در ایران درس میخونه.\",

    \"checkSlang\": true,

    \"normalize\": true,

    \"normalizerParams\": {

        \"text\": \"don't care\",

        \"replaceWildChar\": true,

        \"replaceDigit\": true,

        \"refineSeparatedAffix\": true,

        \"refineQuotationPunc\": false

    },

    \"complexSentence\": true

}'''
print(callApi(url, payload, tokenKey))

[["او","دانشجو","است","."],["در","ایران","درس","می‌خواند","."]]


#  (Spell Correction) تصحیح خطای املایی

In [ ]:
url =  baseUrl + "TextRefinement/SpellCorrector"
payload = u'''{\"text\": \"آب مبات خیلی چوشمزه ست\",

            \"checkSlang\": true,

            \"normalize\": true,

            \"candidateCount\": 3}'''
print(callApi(url, payload, tokenKey))

آب {نبات,مبرات,مباد} {خیلی,خلیلی,تخیلی} {چوش مزه,خوشمزه} {ست,است}


# تبدیل محاوره به رسمی
کلمات محاوره‌ای درون متن به شکل (معادل) رسمی آنها تبدیل می‌شود

In [ ]:
url =  baseUrl + "TextRefinement/FormalConverter"
payload = u'''"میخوام بگم"'''
print(callApi(url, payload, tokenKey))

می‌خواهم بگویم


# تعیین برچسب نقش ادات سخن
این تابع عملیات برچسب زنی نقش (اسم، ضمیر، صفت، قید، فعل، …) کلمات در جمله را انجام می دهد

In [ ]:
url =  baseUrl + "PosTagger/GetPos"
payload = u'"او دانشجو در ایران است"'
result = json.loads(callApi(url, payload, tokenKey))
for phrase in result:
    print("("+phrase['word']+","+phrase['tags']['POS']['item1']+") ")

(او,PRO) 
(دانشجو,N) 
(در,P) 
(ایران,N) 
(است,V) 
(.,) 


# ریشه‌یابی کلمات یا بن‌واژه‌یاب

In [ ]:
url =  baseUrl + "Stemmer/LemmatizeWords2Phrase"
payload = u'["دریانوردانی", "جزایر", "فرشتگان", "تنها", "کتابها"]'
result = json.loads(callApi(url, payload, tokenKey))
for phrase in result:
    print("("+phrase['word']+":"+phrase['firstRoot']+") ")

(دریانوردانی:دریانورد) 
(جزایر:جزیره) 
(فرشتگان:فرشته) 
(تنها:تنها) 
(کتابها:کتاب) 


# NER تشخیص موجودیت های نامدار 

In [ ]:
url =  baseUrl + "NamedEntityRecognition/Detect"
payload = u'"هانی محفوض به تحصیالت خود در دانشگاه اصفهان در ایران ادامه داد"'
result = json.loads(callApi(url, payload, tokenKey))
for phrase in result:
    print("("+phrase['word']+","+phrase['tags']['NER']['item1']+") ")

(هانی,B-PER) 
(محفوض,I-PER) 
(به,O) 
(تحصیالت,O) 
(خود,O) 
(در,O) 
(دانشگاه,B-LOC) 
(اصفهان,I-LOC) 
(در,O) 
(ایران,B-LOC) 
(ادامه,O) 
(داد,O) 


# استخراج مترادف‌ها
این تابع کلمات هم‌معنی (معادل مفهومی) با کلمه ورودی در فرهنگ لغت‌های مختلف را برمی‌گرداند

In [ ]:
url =  baseUrl + "TextSimilarity/ExtractSynonyms"
payload = u"\"دوست\""
print(callApi(url, payload, tokenKey))

["دوست","آشنا","تاش","حبیب","حریف","حمیم","خلیل","یار","همدم","معشوق","رفیق","صدوق","صدیق","محب","محبوب","مصاحب","معاشر","ولی","ولی‌","دوستی","ایاغ","عیاق","رفاقت","ایاق"]


# شناسایی نوع زبان

In [ ]:
url =  baseUrl + "LanguageDetection/Predict"
payload = u'"I go to school"'
print(callApi(url, payload, tokenKey))

en
